# Local Retrieval-Augmented Generation (RAG) Basics


### What is Retrieval-Augmented Generation (RAG)?
RAG combines retrieval (finding relevant information) with generation (producing responses).
This is useful for question-answering, creative text generation, etc.

Think of it like giving an AI assistant access to a library. Instead of only relying on what it memorized during training, it can look up relevant information first, then give you a better answer.

This is useful because:
- LLMs have knowledge cutoffs and can't know everything
- You can add your own documents/data to the system
- Reduces hallucinations by grounding responses in actual sources

# Retrieval

First, let's focus on the retrieval part.

## Embeddings Basics

**What are embeddings?**  
You can think of embeddings as a way to convert words, sentences, or documents into numbers that a computer can understand and compare (vectors in multi-dimensinal matrices). It's like giving each piece of text a unique fingerprint. Similar texts get similar fingerprint, different texts get different fingerprints. 

For example, "Hello" and "Hi" would have very similar embeddings because they mean similar things, while "Hello" and "Banana" would have very different embeddings.

Once we have embeddings, we can do math with meaning! We can calculate how similar two pieces of text are by measuring the distance between their embedding vectors. This is the foundation of semantic search.  

We use Sentence transformers for our embeddings.  
Sentence Transformers (a.k.a. SBERT) is the go-to Python module for accessing, using, and training state-of-the-art embedding and reranker models.  
You can check it out here:  
https://www.sbert.net/index.html 

In [ ]:
from sentence_transformers import SentenceTransformer

embeddings_model = SentenceTransformer('all-MiniLM-L6-v2') # a small general purpose model (80MB)
# this will download the model to your machine, might take a while when you run it for the first time

Next, let's define some sentences and encode them.
Encoding means that we convert our sentences into vectors (embeddings)

In [ ]:
sentences = [
    "Ping me if you need anything.",
    "Let me know if you have any questions.",
    "this email thread was used to train a drone",
]

embedded_sentences = embeddings_model.encode(sentences)

When we print the shape of our embeddings, we see that we have three sentences, each with 384 dimensions.

In [ ]:
print(embedded_sentences.shape)

with our embeddings ready, we can now compute the semantic similarity between all sentences.

In [ ]:
similarities = embeddings_model.similarity(embedded_sentences, embedded_sentences)
print(similarities)

Let's visualize it

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(similarities, cmap="copper")
plt.colorbar()
plt.xticks(range(len(sentences)))
plt.yticks(range(len(sentences)))
plt.title("Similarity Matrix")
plt.show()

Each row and column corresponds to one sentence. The number at position [i, j] shows similarity between sentence i and sentence j.
As you can see, sentences 0 and 1 are somewhat related, but sentence 2 is not related to either of them. The diagonal line shows sentences being fully related to themselves.
Sentence 0 (“ping me”) and Sentence 1 (“let me know”) → Similarity 0.435 (related).
Sentence 0 and Sentence 2 (“train a drone”) → Similarity 0.1194 (unrelated).

## Chunking (splitting text)

If we want to work with longer texts, it's a good idea to split, or *chunk*, them into smaller parts.

Why chunk?
- Embedding models have token limits (can't process infinite text)
- Smaller chunks = more precise retrieval (you get the exact paragraph that answers your question)
- Better performance when searching through large documents
- LLMs work better with focused, relevant context rather than entire documents

Chunking strategies to consider:
- Sentence-based: Good for Q&A, preserves complete thoughts
- Paragraph-based: Better for longer context, maintains topic coherence
- Sliding window: Overlapping chunks to avoid losing context at boundaries
- Semantic chunking: AI-powered chunking that understands topic boundaries

In [ ]:
example_text = """
A measure of uncertainty of an outcome, rather than the perceived lack of order. 
A random sequence of events, symbols or steps often has no order and does not follow an intelligible pattern or combination. 
Randomness exists when some outcomes occur without any order, unpredictably, or by chance. 
These notions are distinct, but they all have a close connection to probability. 
Individual random events are unpredictable, but since they often follow a probability distribution, the frequency of different outcomes over numerous events (or “trials”) is predictable: 
when throwing two dice, the outcome of any particular roll is unpredictable, but a sum of 7 will occur twice as often as 4.
"""

We can write our own chunking functions, for example, by using a maximum character length per chunk.

In [ ]:
def chunk_text_by_length(text, max_length):
    chunks = []
    current_chunk = ""

    for word in text.split():
        # Check if adding the next word exceeds the max length
        if len(current_chunk) + len(word) + 1 <= max_length:
            current_chunk += (word + " ")
        else:
            chunks.append(current_chunk.strip())
            current_chunk = word + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

In [ ]:
print(chunk_text_by_length(example_text, 50))

As you can see, that might not be the best approach, as a lot of meaning is lost between chunks.  
Another approach is to split the text at periods, so that each chunk ideally contains a single sentence.

In [ ]:
def chunk_text_by_period(text):
    sentences = text.split('.')
    chunks = []

    for sentence in sentences:
        sentence = sentence.strip()
        if sentence:
            chunks.append(sentence + '.')

    return chunks

In [ ]:
print(chunk_text_by_period(example_text))

This is already an improvement; however, it can result in chunks of varying length. In many cases, it might be better to have larger chunks, such as paragraphs. Since text can take many forms, there is no one-size-fits-all solution.
There are some prewritten methods that you can use to chunk text that we will cover later.

In [ ]:
# here add chunking functions

chunks = chunk_text_by_period(example_text)
print(chunks)

## Vector Databases (Chroma DB)

We use Chroma as a vector database. When working with a large number of embeddings, it's a good idea to store them in a database. Chroma is an open-source vector search and retrieval database that you can easily deploy locally.  
You can check it out here: https://www.trychroma.com/  

Why use a vector database?
- Speed: Optimized for similarity search across millions of vectors
- Persistence: Your embeddings are saved between sessions
- Scalability: Can handle large document collections efficiently
- Metadata: Store additional info with each chunk (source, date, etc.)

In [ ]:
import chromadb

# Create client and collection
client = chromadb.PersistentClient() # we use a persistent client to store our db 
# client.delete_collection(name="example_collection_rag") # uncomment in case you want to start over
collection = client.get_or_create_collection("example_collection_rag")

embedded_chunks = embeddings_model.encode(chunks)

# generic ids for the chunks
ids = [f"chunk_{i+1}" for i in range(len(chunks))]

# Add embeddings manually
collection.add(
    embeddings=embedded_chunks,
    documents=chunks,
    ids=ids
)

## Similarity Search 

In [ ]:
query = "What is random?"
query_embedding = embeddings_model.encode(query)

results = collection.query(
    query_embeddings=[query_embedding],
    n_results=2
)

print("Results:", results)

## Ollama 
Ollama is a local runtime for working with LLMs on your local machine. We'll use it to simply download models and generate text based on our retrieved documents.  
Essentially, it's a wrapper around https://github.com/ggml-org/llama.cpp but we'll stick with ollama for now since it's very easy to use.
You can download Ollama here: https://ollama.com/download  

Why Ollama?
- Privacy: Everything runs locally, no data sent to external APIs
- Cost: Free to use once you have the hardware
- Speed: No network latency for inference
- Customization: Full control over model parameters  

How it works:  

You can serve ollama after you installed it by typing `ollama serve` in your terminal. Then you can use a second terminal window to interact with it.

If you run the GUI, it automatically serves all downloaded models on your PC.

You can check which models you already downloaded with `ollama list`  

To run a model from your terminal, simply `ollama run` and select a model for example `ollama run gemma3:1b`  if you haven't downloaded it yet it will do so automatically.  

to stop chatting with a model, type `/bye`.

To use ollama in python, we need to `pip install ollama` to install the bindings.  
Then we can test it like this:

In [ ]:
from ollama import chat
from ollama import ChatResponse

# the query we send to the LLM
query = "What is randomness?"

# this is the structure expected by the ollama API
response: ChatResponse = chat(model='gemma3:1b', messages=[
  {
    'role': 'user',
    'content': query,
  },
])

print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

## Connecting Retrieval and Generation

This is where RAG comes together! 
 
- **R**etrieve relevant chunks based on the user's question
- **A**ugment the prompt by including this context
- **G**enerate a response using both the context and the LLM's knowledge

In [ ]:
def query_rag(query, top_k=1):
    
    # RETRIEVE
    query_embedding = embeddings_model.encode(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    context = "\n".join(results['documents'][0])
    
    # AUGMENT
    # Prompt the local Ollama model
    prompt = f"Using the following context, answer the question:\n\nContext:\n{context}\n\nQuestion:\n{query}\n"

    print(f"Prompt:\n{prompt}")
    
    # GENERATE
    response: ChatResponse = chat(
        model='gemma3:1b', 
        messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ]
        )
    return response.message.content

# Test query
query = "What is random?"
response = query_rag(query)

print(f"Response:\n{response}")

## Parsing PDF's 

### PyMuPDF & Tesseract

PyMuPDF is a simple library for parsing PDF's.  
To perform Optical Character Recognition (OCR) on scanned files or images of text, we'll also install Tesseract.   
The results may vary, as these libraries are fast and easy to install, but not always perfect at text extraction.

You can see how to install tesseract on your OS here:  
https://tesseract-ocr.github.io/tessdoc/Installation.html  
https://github.com/UB-Mannheim/tesseract/wiki  

To install PyMuPDF: 
`pip install pymupdf`  

https://github.com/pymupdf/PyMuPDF  
https://github.com/tesseract-ocr/tesseract  

In [ ]:
import pymupdf

doc = pymupdf.open("/Users/c/Desktop/pdf_extraction/PDF/138641610-Ways-of-Seeing.pdf") # open a document

text_pages = []

for page in doc: # iterate the document pages
  text = page.get_text() # get plain text encoded as UTF-8
  text_pages.append(text)

text_all = "\n".join(text_pages)

print(text_all)
print(len(text_all))

doc.close()

In [ ]:
print(text_pages[5])

Chances are your computer already performed OCR on your PDF or the text is actually embedded in your PDF in which case the text extraction should happen rather quickly.  
In case it's not, or when you want to run OCR yourself PyMuPDF will use tesseract for OCR. There are also other LLM-assisted libraries that will perform better at this task but these usually require a decent GPU to run.

In [ ]:
doc = pymupdf.open("/Users/c/Desktop/pdf_extraction/PDF/138641610-Ways-of-Seeing.pdf")

# force OCR for the first 5 pages
for i in range(min(5, doc.page_count)):
    page = doc[i]
    tp = page.get_textpage_ocr(dpi=400, language="eng", full=True)
    txt = tp.extractText("text")
    print(f"\n--- page {i+1} OCR text ---\n", txt[:500])

doc.close()

There is an additional package called pymupdf4llm that we can use to convert our pdf to markdown, which is much better suited for RAG than regular text files, as we can use the structure for chunking.

In [ ]:
import pymupdf4llm

md_text = pymupdf4llm.to_markdown("/Users/c/Desktop/pdf_extraction/PDF/138641610-Ways-of-Seeing.pdf")

print(md_text)

In [ ]:
# Now work with the markdown text, e.g. store as a UTF8-encoded file
import pathlib
pathlib.Path("/Users/c/Desktop/pdf_extraction/output.md").write_bytes(md_text.encode())

### Docling
Docling is a document parser with local AI models for OCR.
It will perform vastly better than our approach above, but depending on your machine it will also take significantly longer.  
It is still a good solution for local parsing and it also converts text to various formats and even chunks it for LLM use.
The Docling AI models are relatively small and should work fine on a rather recent macbook or similar.  
This will download several models for detection and recognition to your PC, so make sure you have some space left on your hard drive. 
The Docling project was started by the AI for knowledge team at IBM Research Zurich.  

`pip install docling`  

https://github.com/docling-project/docling  

In [ ]:
from docling.document_converter import DocumentConverter

source = "/Users/c/Desktop/pdf_extraction/PDF/138641610-Ways-of-Seeing.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
print(result.document.export_to_markdown())
result.save_as_markdown("/Users/c/Desktop/pdf_extraction/out-md.md")


In [ ]:
# accelerated workflow for macs
# pip install mlx-vlm

from docling.datamodel import vlm_model_specs
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import VlmPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.pipeline.vlm_pipeline import VlmPipeline

source = "/Users/c/Desktop/pdf_extraction/PDF/138641610-Ways-of-Seeing.pdf"  # document per local path or URL

pipeline_options = VlmPipelineOptions(
    vlm_options=vlm_model_specs.SMOLDOCLING_MLX,
)

converter = DocumentConverter(
    format_options={
        InputFormat.PDF: PdfFormatOption(
            pipeline_cls=VlmPipeline,
            pipeline_options=pipeline_options,
        ),
    }
)

doc = converter.convert(source=source).document
print(doc.export_to_markdown())

In [ ]:
doc.save_as_markdown("/Users/c/Desktop/pdf_extraction/out-markdown.md")

### Chunking

We use docling for chunking, however there are many other libraries, such as LangChain or Sentence Transformers that could also do this.  
There are various approaches to chunking and the best approach depends on your data and LLM pipeline.  
The docling HybridChunker is pretty versatile and since we already installed the package, let's use it to chunk our document.

In [ ]:
from docling.chunking import HybridChunker
from docling.document_converter import DocumentConverter

DOC_SOURCE = "/Users/c/Desktop/pdf_extraction/out-markdown.md"

doc = DocumentConverter().convert(source=DOC_SOURCE).document

chunker = HybridChunker()
chunk_iter = chunker.chunk(dl_doc=doc)

In [ ]:
for i, chunk in enumerate(chunk_iter):
    print(f"=== {i} ===")
    print(f"chunk.text:\n{f'{chunk.text[:300]}…'!r}")

    enriched_text = chunker.contextualize(chunk=chunk)
    print(f"chunker.contextualize(chunk):\n{f'{enriched_text[:300]}…'!r}")

    print()

### Embedding the chunks

We use the same chroma setup as above and use it to encode our chunks.

In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create client and collection
client = chromadb.PersistentClient() # we use a persistent client to store our db 
# client.delete_collection(name="example_collection")
collection = client.create_collection("example_collection")

In [ ]:
DOC_SOURCE = "/Users/c/Desktop/pdf_extraction/out-markdown.md"

doc = DocumentConverter().convert(source=DOC_SOURCE).document

chunker = HybridChunker()
chunk_iter = chunker.chunk(dl_doc=doc)

for i, chunk in enumerate(chunk_iter):

    enriched_text = chunker.contextualize(chunk=chunk)
    
    embeddings = embeddings_model.encode(enriched_text)

    # Optionally, add metadata
    metadata = {"chunk_id": i}

    # Add to collection
    collection.add(
        embeddings=embeddings,
        documents=enriched_text,
        metadatas=metadata,
        ids=[f"chunk_{i}"]
    )

    print(f"Added chunk {i}")

Now we can use the same approach as above to query our LLM and augment it with our retrieved data.

In [ ]:
from ollama import chat
from ollama import ChatResponse
# make sure to 'ollama serve'

def query_rag(query, top_k=4):
    
    query_embedding = embeddings_model.encode(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    context = "\n".join(results['documents'][0])
    
    # Prompt the local Ollama model
    prompt = f"Using the following context, answer the question:\n\nContext:\n{context}\n\nQuestion:\n{query}\n"

    print(f"Prompt:\n{prompt}")
    
    response: ChatResponse = chat(
        model='gemma3:1b', 
        messages=[
            {
                'role': 'user',
                'content': prompt,
            },
        ]
        )
    return response.message.content

# Test query
query = "how to look at art?"
response = query_rag(query)

print(f"Response:\n{response}")